<a href="https://colab.research.google.com/github/surkovvv/TinkoffML_Fall2021/blob/main/Task8Surkov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-12-12 17:42:41--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 173.194.213.139, 173.194.213.102, 173.194.213.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.213.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pkn350mv0btmrj4q17nm0pri4b94lqqc/1639330950000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2021-12-12 17:42:42--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pkn350mv0btmrj4q17nm0pri4b94lqqc/1639330950000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [5]:
df.sample(4)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
8610,8610,2014-11-25,32,18.0,2.0,9.0,1.0,1973.0,1.0,6.0,2.0,Investment,Pechatniki,1.844581e+07,83369,0.030697,0.428826,4615,2936.0,5,5011,5791.0,5,0,730.0,0,0,4,5,no,0,2,8,no,no,no,yes,no,no,no,...,642.86,1107.14,875.00,3,5,14,8,1,0,0,2,11,0,0,24,7,12.40,24.71,40,1701997,47,1865502,167,680.13,1153.85,916.99,11,47,57,35,14,3,0,11,31,0,8,67,14,5200000
6610,6610,2015-02-27,42,41.0,21.0,1.0,1.0,2015.0,1.0,1.0,1.0,OwnerOccupier,Losinoostrovskoe,5.361327e+06,80917,0.197068,0.036122,4169,1926.0,3,4378,6839.0,4,0,1036.0,1,0,11,3,no,0,1,0,no,no,no,yes,no,no,no,...,647.17,1094.34,870.75,11,16,19,14,4,0,0,9,8,0,1,38,1,29.14,7.54,20,366279,28,3372456,119,633.01,1072.82,852.91,16,39,33,21,9,1,0,11,14,1,3,61,3,7497000
19077,19077,2014-01-14,51,30.0,4.0,17.0,1.0,2004.0,2.0,9.0,3.0,Investment,Zapadnoe Degunino,7.632940e+06,78810,0.051844,0.437885,5519,1503.0,3,5735,5782.0,3,0,1084.0,0,0,3,2,no,0,5,3,no,no,no,yes,no,no,no,...,805.08,1330.51,1067.80,3,10,21,22,3,3,0,4,6,0,0,24,1,13.26,20.26,43,1151677,39,1290812,173,705.13,1189.10,947.12,17,40,60,42,9,5,0,11,16,0,0,58,4,2000000
2113,2113,2015-02-12,42,NaN,12.0,0.0,1.0,NaN,1.0,0.0,NaN,OwnerOccupier,Poselenie Sosenskoe,6.677245e+07,9553,0.336177,0.072158,656,NaN,0,629,NaN,0,0,NaN,0,0,1,0,no,0,0,1,no,no,no,no,no,yes,no,...,1375.00,2250.00,1812.50,0,0,1,1,1,1,0,0,5,0,0,1,0,32.84,11.29,3,108859,4,335876,26,760.00,1260.00,1010.00,1,10,6,5,2,2,0,3,13,0,0,6,3,3900000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [6]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]

timestamp = df['timestamp']

df.drop(columns=drop_columns, inplace=True)

target = df["price"]
df = df.drop("price", axis=1)

cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]

num_columns = list(set(df.columns).difference(set(cat_columns)))

In [7]:
df.isna().sum().sort_values().iloc[-52:]

school_km                                   0
metro_km_walk                              16
railroad_station_walk_min                  16
metro_min_walk                             16
ID_railroad_station_walk                   16
railroad_station_walk_km                   16
floor                                     113
prom_part_5000                            118
cafe_sum_5000_min_price_avg               196
cafe_sum_5000_max_price_avg               196
cafe_avg_price_5000                       196
cafe_sum_3000_min_price_avg               641
cafe_sum_3000_max_price_avg               641
cafe_avg_price_3000                       641
cafe_sum_2000_min_price_avg              1134
cafe_sum_2000_max_price_avg              1134
cafe_avg_price_2000                      1134
cafe_sum_1500_min_price_avg              2769
cafe_avg_price_1500                      2769
cafe_sum_1500_max_price_avg              2769
build_count_after_1995                   3175
build_count_slag                  

# О НЕТ! У НАС В ДАННЫХ ПРОПУСКИ 🆘 
# SPOILER: я не знал, что тут по-умному обрабатывать-мучаться не стоит, поэтому код ниже это круто, полезно наверное, но будет закомменчен

In [8]:
"""
void_columns = df.columns[df.isna().sum() > 2000] # больше 10% пропусков, можно подкрутить как вверх, так и вниз 
print("Колонок до первой чистки : ", df.columns.shape[0])
df.drop(columns=void_columns, inplace=True)
cat_columns = list(set(cat_columns).difference(set(void_columns)))
num_columns = list(set(num_columns).difference(set(void_columns)))
"""

'\nvoid_columns = df.columns[df.isna().sum() > 2000] # больше 10% пропусков, можно подкрутить как вверх, так и вниз \nprint("Колонок до первой чистки : ", df.columns.shape[0])\ndf.drop(columns=void_columns, inplace=True)\ncat_columns = list(set(cat_columns).difference(set(void_columns)))\nnum_columns = list(set(num_columns).difference(set(void_columns)))\n'

In [9]:
# В остальных просто заменим пропуски - медианным значением и самым популярным классом, в зависимости от типа данных
"""
cols_with_nans = df.columns[df.isna().sum() > 0]
for col_name in cols_with_nans:
    if col_name in cat_columns or df[col_name].dtype == "object":
        df[col_name].fillna(df[col_name].mode(), inplace=True)
    else :
        df[col_name].fillna(df[col_name].median(), inplace=True)
df["ID_railroad_station_walk"].fillna(df['ID_railroad_station_walk'].median(), inplace=True)
"""

'\ncols_with_nans = df.columns[df.isna().sum() > 0]\nfor col_name in cols_with_nans:\n    if col_name in cat_columns or df[col_name].dtype == "object":\n        df[col_name].fillna(df[col_name].mode(), inplace=True)\n    else :\n        df[col_name].fillna(df[col_name].median(), inplace=True)\ndf["ID_railroad_station_walk"].fillna(df[\'ID_railroad_station_walk\'].median(), inplace=True)\n'

In [10]:
#df.isna().sum().sort_values()

In [11]:
#print("Колонок после первой чистки : ", df.columns.shape[0])

## Ура, пропусков нема

In [12]:
#df[cat_columns].nunique().sort_values(ascending=False)

# О НЕТ! У НАС ЕСТЬ КАТЕГОРИАЛЬНЫЕ КОЛОНКИ С БОЛЬШИМ КОЛИЧЕСТВОМ УНИКАЛЬНЫХ ЗНАЧЕНИЙ!(а значит никаких OHE) 🆘🆘

In [13]:
# попробуем просто удалить 4 колонки
"""
bad_4 = df[cat_columns].nunique().sort_values(ascending=False).index[:4]
df.drop(columns=bad_4, inplace=True)
cat_columns = list(set(cat_columns).difference(set(bad_4)))
"""

'\nbad_4 = df[cat_columns].nunique().sort_values(ascending=False).index[:4]\ndf.drop(columns=bad_4, inplace=True)\ncat_columns = list(set(cat_columns).difference(set(bad_4)))\n'

In [14]:
#df[cat_columns].nunique().sort_values(ascending=False)

# Отлично, вроде вышло неплохо

### попробуем самую тупую обработку пропусков - просто заменим пропуски на средние значений(медиану и моду, если быть точнее)

In [8]:
for col_name in cat_columns:
    if any(df[col_name].isna()):
        df[col_name].fillna(df[col_name].mode().values[0], inplace=True)
for col_name in num_columns:
    if any(df[col_name].isna()):
        df[col_name].fillna(df[col_name].median(), inplace=True)

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
scaler = StandardScaler()
df_catboost = df
df_catboost[num_columns] = scaler.fit_transform(df_catboost[num_columns])
encoded_cat = pd.get_dummies(df[cat_columns])
df = df.drop(columns=cat_columns)
df = df.join(encoded_cat)
df.loc[:, :] = scaler.fit_transform(df) 

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, target, 
                                                    train_size=0.8, random_state=42)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

base_model = DecisionTreeRegressor().fit(X_train, y_train)

Проверьте качество на отложенной выборке.

In [12]:
y_base = base_model.predict(X_test)
print("RMSE Score on base model {:.4f}".format(mean_squared_error(y_test, y_base)**0.5))

RMSE Score on base model 3851607.5172


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [13]:
month_year = (timestamp.dt.month + timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)
df_catboost["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (timestamp.dt.weekofyear + timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)
df_catboost["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitch_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [14]:
df["month"] = timestamp.dt.month
df['day_of_week'] = timestamp.dt.dayofweek
df_catboost["month"] = timestamp.dt.month
df_catboost['day_of_week'] = timestamp.dt.dayofweek

In [15]:
df['floors_ratio'] = df["floor"] / df["max_floor"]
df["kitchen_ratio"] = df["kitch_sq"] / df["full_sq"]
df_catboost['floors_ratio'] = df_catboost["floor"] / df_catboost["max_floor"]
df_catboost["kitchen_ratio"] = df_catboost["kitch_sq"] / df_catboost["full_sq"]

In [16]:
float_cols = df_catboost[cat_columns].select_dtypes("float").columns
df_catboost.loc[:, float_cols] = df_catboost.loc[:, float_cols].astype(int)

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df, target, 
                                                    train_size=0.8, random_state=42)
X_trainc, X_testc, y_trainc, y_testc = train_test_split(df_catboost, target, 
                                                    train_size=0.8, random_state=42)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

## 1 - DecisionTree без подбора гиперпараметров

In [25]:
dt = DecisionTreeRegressor().fit(X_train, y_train)
y_pred1 = dt.predict(X_test)
print("RMSE Score on base model {:.4f}".format(mean_squared_error(y_test, y_pred1)**0.5))

RMSE Score on base model 3794778.5576


## 2 - DecisionTree с подбором гиперпараметров

In [26]:
params = {
    "max_depth" : np.logspace(1,6, 6, base=2),
    "min_samples_leaf" : np.arange(1, 100, 10)
}
dth = DecisionTreeRegressor()
grid = GridSearchCV(estimator=dth, param_grid=params, scoring="neg_root_mean_squared_error")
grid.fit(X_train, y_train)
grid.best_params_

{'max_depth': 8.0, 'min_samples_leaf': 51}

In [27]:
dth = DecisionTreeRegressor(max_depth=8, min_samples_leaf=10)
dth.fit(X_train, y_train)
y_pred2 = dth.predict(X_test)
print("RMSE Score on tunned DT {:.4f}".format(mean_squared_error(y_test, y_pred2)**0.5))

RMSE Score on tunned DT 3014222.9253


## 3 - RandomForest без гиперпараметров

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred3 = rf.predict(X_test)


In [19]:
print("RMSE Score on RF {:.4f}".format(mean_squared_error(y_test, y_pred3), squared=False))

RMSE Score on RF 6823062590666.9980


## 4 - RandomForest с гиперпараметрами

In [29]:
params = {
    "n_estimators" : [100, 300],
    "max_depth" : [2, 4, 8, 16],
    "min_samples_leaf" : np.arange(1, 40, 10)
}
rfh = RandomForestRegressor(n_jobs=-1)
grid = GridSearchCV(estimator=rfh, param_grid=params, scoring="neg_root_mean_squared_error")
grid.fit(X_train, y_train)
grid.best_params_

{'max_depth': 16, 'min_samples_leaf': 1, 'n_estimators': 100}

In [19]:
params = {"max_depth" : 16,
          "min_samples_leaf" : 1,
          "n_estimators" : 100}
rfh = RandomForestRegressor(**params)
rfh.fit(X_train, y_train)
rfh_pred = rfh.predict(X_test)
print("RMSE Score on RF {:.4f}".format(mean_squared_error(y_test, rfh_pred), squared=False))

RMSE Score on RF 6972659555814.5195


In [20]:
pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [21]:
import catboost

catmodel = catboost.CatBoostRegressor()
catmodel.fit(X_train, y_train, verbose=False)
y_pred = catmodel.predict(X_test)
print("CatBoost RMSE score {:.4f}".format(mean_squared_error(y_pred, y_test, squared=False)))

CatBoost RMSE score 2499705.0478


In [32]:
catmodel = catboost.CatBoostRegressor(cat_features=cat_columns)
catmodel.fit(X_trainc, y_trainc, cat_features=cat_columns, verbose=False)
y_pred = catmodel.predict(X_testc)
print("CatBoost RMSE score with impemented cat features {:.4f}".format(mean_squared_error(y_pred, y_test, squared=False)))

CatBoost RMSE score with impemented cat features 2485177.8225


In [ ]:
from catboost import Pool, cv
from google.colab import output
output.enable_custom_widget_manager()

params = {"depth": 2,
          "loss_function": "RMSE",
          "verbose": False,
          "cat_features" : cat_columns}

cv_dataset = Pool(data=X_trainc,
                  label=y_trainc,
                  cat_features=cat_columns)

scores = cv(cv_dataset,
            params,
            fold_count=2, 
            plot="False")

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5, 7]}

catmodel = catboost.CatBoostRegressor(cat_features=cat_columns)

grid_search_result = catmodel.grid_search(grid, X=X_trainc, y=y_trainc)

In [35]:
grid_search_result["params"]

{'depth': 8, 'l2_leaf_reg': 7, 'learning_rate': 0.1}

Таким образом, catboost лучше делает предобработку категориальных признаков, чем просто OHE

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [24]:
X_invest = df[df_catboost["product_type"] == "Investment"].drop(columns=["product_type_Investment",
                                                                "product_type_OwnerOccupier"])
X_owner = df[df_catboost["product_type"] == "OwnerOccupier"].drop(columns=["product_type_Investment",
                                                                "product_type_OwnerOccupier"])
X_investc = df_catboost[df_catboost["product_type"] == "Investment"].drop(columns=["product_type"])
X_ownerc = df_catboost[df_catboost["product_type"] == "OwnerOccupier"].drop(columns=["product_type"])

y_invest = target[df_catboost["product_type"] == "Investment"]
y_owner = target[df_catboost["product_type"] == "OwnerOccupier"]

In [25]:
X_invest_train, X_invest_test, y_invest_train, y_invest_test = train_test_split(X_invest, 
                                                                                y_invest, 
                                                                                train_size=0.8, 
                                                                                random_state=42)

X_investc_train, X_investc_test, y_investc_train, y_investc_test = train_test_split(X_investc,
                                                                                    y_invest, 
                                                                                    train_size=0.8, 
                                                                                    random_state=42)
X_ownerc_train, X_ownerc_test, y_ownerc_train, y_ownerc_test = train_test_split(X_ownerc,
                                                                                y_owner, 
                                                                                train_size=0.8, 
                                                                                random_state=42)
X_owner_train, X_owner_test, y_owner_train, y_owner_test = train_test_split(X_owner,
                                                                            y_owner, 
                                                                            train_size=0.8, 
                                                                            random_state=42)

In [26]:
cat_columns.remove("product_type")

In [36]:
catmodel1 = catboost.CatBoostRegressor(cat_features=cat_columns, **grid_search_result["params"])

catmodel1.fit(X_investc_train, y_investc_train, cat_features=cat_columns, verbose=False)

In [37]:
catmodel2 = catboost.CatBoostRegressor(cat_features=cat_columns, **grid_search_result["params"]) 
catmodel2.fit(X_ownerc_train, y_ownerc_train, cat_features=cat_columns, verbose=False)
y_pred_owner = catmodel2.predict(X_ownerc_test)

In [38]:
y_pred_invest= catmodel1.predict(X_investc_test)
y_pred_owner = catmodel2.predict(X_ownerc_test)

In [39]:
print("Ensemble №1", mean_squared_error(np.concatenate((y_pred_invest, y_pred_owner)),
                                        np.concatenate((y_invest_test, y_owner_test)), 
                                        squared=False))

Ensemble №1 2504020.6250809645


Возник вопрос - можно ли не использовать 2 модели, а обучиться на первой, предсказать, потом зановой обучиться на той же?

# По итогу, либо я сделал что-то неправильно, либо в действительности один catboost-regressor работает лучше, чем два отдельно обученных (может быть признак product_type является важным)

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
from sklearn.linear_model import HuberRegressor

cat = catboost.CatBoostRegressor(**grid_search_result["params"])
cat.fit(X_invest_train, y_invest_train)
cat_pred = cat.predict(X_invest_test)


In [45]:
huber = HuberRegressor(max_iter=1000)
huber.fit(X_invest_train, y_invest_train)
huber_pred = huber.predict(X_invest_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [46]:
for w in np.arange(0, 1.1, 0.1):
    y_pred = w * cat_pred + (1-w) * huber_pred
    print("With w1={}, w2={} result is {}".format(w, 1-w, 
                                                  mean_squared_error(y_pred, y_invest_test, 
                                                                     squared=False)))

With w1=0.0, w2=1.0 result is 3482584.636264827
With w1=0.1, w2=0.9 result is 3391765.2972590397
With w1=0.2, w2=0.8 result is 3309715.574471853
With w1=0.30000000000000004, w2=0.7 result is 3237102.3779578493
With w1=0.4, w2=0.6 result is 3174573.309301131
With w1=0.5, w2=0.5 result is 3122734.19556354
With w1=0.6000000000000001, w2=0.3999999999999999 result is 3082124.4759783093
With w1=0.7000000000000001, w2=0.29999999999999993 result is 3053192.2612202503
With w1=0.8, w2=0.19999999999999996 result is 3036271.389680746
With w1=0.9, w2=0.09999999999999998 result is 3031562.9943777234
With w1=1.0, w2=0.0 result is 3039123.836771871


Получилось не очень - уж больно неточные ответы вышли у модели Хубера